In [ ]:
def graphclass_default(X, Y, Xtest, Ytest, Adj_list, type = c("intersection", "union", "groups", "fusion"), lambda = 0, 
                       rho = 0, gamma = 1e-5, 
                       params = NULL, id = "", verbose = F, D = NULL,
                       Groups = NULL,
                       G_penalty_factors):
    g1_results = {}    
    g1_results['lambda'] = lambda
    g1_results['rho'] = rho
    g1_results['gamma'] = gamma
    
    if(!is.null(X)) {
    # ncols = N*(N-1)/2
    NODES = (1+sqrt(1+8*ncol(X)))/2
      }else{
    NODES <- ncol(Adj_list[[1]])
    X = t(sapply(Adj_list, function(A) A[upper.tri(A)]))
    }
    
    Yoriginal = Y
    Y = (Y-min(Y))/(max(Y)-min(Y))
    Y = 2*Y-1
    
    
    
#===========================================================================#
UCONVERTED R CODE


    NODES <- ncol(Adj_list[[1]])
    X <- t(sapply(Adj_list, function(A) A[upper.tri(A)]))
  }
  Yoriginal <- Y
  Y <- as.numeric(Y)
  Y <- (Y-min(Y))/(max(Y)-min(Y))
  Y <- 2*Y-1
  gl_results$Ypos_label <- unique(Yoriginal[Y==1])[1]
  gl_results$Yneg_label <- unique(Yoriginal[Y==-1])[1]
  table(Y, Yoriginal)
  # Normalize X for numerical stability
  alpha_normalization <- max(apply(X, 2, function(v) sd(v)))
  X <- X/alpha_normalization
  lambda1 <- lambda*rho/alpha_normalization
  lambda2 <- lambda/alpha_normalization
  gamma <- gamma/alpha_normalization
  # Create D
  if(is.null(D)) {
    if(type=="intersection"| type=="union") {
      D <- construct_D(NODES)  
    }else{
      if(type=="fusion")
        D <- construct_D_fusion(NODES)
      else{if(type=="groups") {
        require(Matrix)
        D_list <- lapply(Groups, function(G) {
          D <- Matrix(0, nrow = ncol(X), ncol = ncol(X))
          Matrix::diag(D)[G] <- 1
          D
          })
        if(is.null(G_penalty_factors)) {
          G_penalty_factors <- rep(1/length(D_list), length(D_list))
        }
      }else{
        stop("The value of type should be one 
             between \"intersection\", \"union\" or \"groups\".")
          }
        }
      }
    }
  # Check which method to use
  # Intersection penalty -----------------------------------------------
  if(type=="intersection") {
    beta_start <- rep(0,NODES*(NODES-1)/2)
    b_start <- 0
    MAX_ITER <- 300;    CONV_CRIT <- 1e-05;   MAX_TIME = Inf
    # Check params
    if(!is.null(params)){
      if(!is.null(params$beta_start))
        beta_start <- params$beta_start*alpha_normalization
      if(!is.null(params$b_start))
        b_start <- params$b_start
      if(!is.null(params$MAX_ITER))
        MAX_ITER <- params$MAX_ITER
      if(!is.null(params$CONV_CRIT))
        CONV_CRIT <- params$CONV_CRIT
        
    }
    gl = logistic_group_lasso_ridge(X, Y, D,
                                    lambda1 = lambda1, 
                                    lambda2 = lambda2,
                                    gamma = gamma,
                                    id, verbose, 
                                    beta_start = beta_start, 
                                    b_start = b_start,
                                    NODES = NODES, 
                                    MAX_ITER = MAX_ITER, 
                                    CONV_CRIT = CONV_CRIT, 
                                    MAX_TIME = MAX_TIME)
    gl_results$beta = gl$best_beta/alpha_normalization
    gl_results$b = gl$best_b
    
    # Train fitting
    Yfit <- alpha_normalization*(X%*%gl_results$beta) + gl_results$b
    gl_results$Yfit <- exp(Yfit)/(1+exp(Yfit))
    gl_results$train_error <- 1- sum(diag(table(sign(Yfit),sign(Y))))/length(Y)
    class(gl_results) <- "graphclass"
    # Test fitting
    if(!is.null(Xtest)) {
      predict.graphclass(gl_results, Xtest)
      Yfit_test <- Xtest%*%gl_results$beta + gl_results$b
      Ypred <- 1*(Yfit_test>0) -1*(Yfit_test<=0)
      gl_results$Ypred <- predict.graphclass(gl_results, Xtest)
      gl_results$Yfit_test <- predict.graphclass(gl_results, Xtest, type = "prob")
      if(!is.null(Ytest)) {
        gl_results$test_error = predict.graphclass(gl_results, Xtest, type = "error", Ytest = Ytest)
      } 
    }
    gl_results$type = "intersection"
    gl_results$active_nodes = active_nodes(gl_results$beta, NODES = NODES)
    gl_results$subgraph_active_edges_rate = active_edges_rate(gl_results$beta, NODES = NODES)
    gl_results$nonzeros_percentage = sum(gl_results$beta!=0)/length(gl_results$beta)
    return(gl_results)
    
  }else{if(type=="groups") {
    # Check params
    beta_start <- rep(0,NODES*(NODES-1)/2)
    b_start <- 0
    MAX_ITER <- 300;    CONV_CRIT <- 1e-05;   MAX_TIME = Inf
    # Check params
    if(!is.null(params)){
      if(!is.null(params$beta_start))
        beta_start <- params$beta_start*alpha_normalization
      if(!is.null(params$b_start))
        b_start <- params$b_start
      if(!is.null(params$MAX_ITER))
        MAX_ITER <- params$MAX_ITER
      if(!is.null(params$CONV_CRIT))
        CONV_CRIT <- params$CONV_CRIT
    }
    print(CONV_CRIT)
    print(MAX_ITER)
    gl = logistic_group_lasso_ridge_groups(X, Y, Groups,
                                    lambda1 = lambda1, 
                                    lambda2 = lambda2,
                                    G_penalty_factors = G_penalty_factors,
                                    gamma = gamma,
                                    id, verbose, 
                                    beta_start = beta_start, 
                                    b_start = b_start,
                                    NODES = NODES, 
                                    MAX_ITER = MAX_ITER, 
                                    CONV_CRIT = CONV_CRIT, 
                                    MAX_TIME = MAX_TIME)
    gl_results$beta = gl$best_beta/alpha_normalization
    gl_results$b = gl$best_b
    
    # Train fitting
    Yfit <- alpha_normalization*(X%*%gl_results$beta) + gl_results$b
    gl_results$Yfit <- exp(Yfit)/(1+exp(Yfit))
    gl_results$train_error <- 1- sum(diag(table(sign(Yfit),sign(Y))))/length(Y)
    class(gl_results) <- "graphclass"
    # Test fitting
    if(!is.null(Xtest)) {
      predict.graphclass(gl_results, Xtest)
      Yfit_test <- Xtest%*%gl_results$beta + gl_results$b
      Ypred <- 1*(Yfit_test>0) -1*(Yfit_test<=0)
      gl_results$Ypred <- predict.graphclass(gl_results, Xtest)
      gl_results$Yfit_test <- predict.graphclass(gl_results, Xtest, type = "prob")
      if(!is.null(Ytest)) {
        gl_results$test_error = predict.graphclass(gl_results, Xtest, type = "error", Ytest = Ytest)
      } 
    }
    gl_results$type = "intersection"
    gl_results$active_nodes = active_nodes(gl_results$beta, NODES = NODES)
    gl_results$subgraph_active_edges_rate = active_edges_rate(gl_results$beta, NODES = NODES)
    gl_results$nonzeros_percentage = sum(gl_results$beta!=0)/length(gl_results$beta)
    return(gl_results)
    
  }else{if(type == "union"){
    # Union penalty ---------------------------------------------------
    # Check params
    beta_start <- rep(0,NODES*(NODES-1))
    b_start <- 0
    MAX_ITER <- 300;    CONV_CRIT <- 1e-05;   MAX_TIME = Inf
    # Check params
    if(!is.null(params)){
      if(!is.null(params$beta_start))
        beta_start <- params$beta_start*alpha_normalization
      if(!is.null(params$b_start))
        b_start <- params$b_start
      if(!is.null(params$MAX_ITER))
        MAX_ITER <- params$MAX_ITER
      if(!is.null(params$CONV_CRIT))
        CONV_CRIT <- params$CONV_CRIT
      
    }
    gl = logistic_union_group_lasso_ridge(X, Y, D,
                                          lambda1 = lambda1, 
                                          lambda2 = lambda2, 
                                          gamma = gamma,
                                          id, verbose, 
                                          beta_start = beta_start, 
                                          b_start = b_start,
                                          NODES = NODES, 
                                          MAX_ITER = MAX_ITER, 
                                          CONV_CRIT = CONV_CRIT, 
                                          MAX_TIME = MAX_TIME)
    gl_results$beta = gl$best_beta/alpha_normalization
    gl_results$b = gl$best_b
    # Train fitting
    Yfit <- alpha_normalization*(X%*%(crossprod(D,gl_results$beta ))) + gl$b
    gl_results$Yfit <- exp(Yfit)/(1+exp(Yfit))
    gl_results$train_error <- 1-sum(diag(table(as.vector(sign(gl_results$Yfit-0.5)),sign(Y))))/length(Y)
    # Test fitting
    if(!is.null(Xtest)) {
      Yfit_test <- Xtest%*%crossprod(D,gl_results$beta) + gl_results$b
      gl_results$Yfit_test <- exp(Yfit_test)/(1+exp(Yfit_test))
      if(is.factor(Yoriginal)) {
        gl_results$Ypred <- factor(Ypred)
        levels(gl_results$Ypred) <- levels(Yoriginal)
      }
      if(!is.null(Ytest)) {
        levels(Ytest) <- c(-1, 1)
        gl_results$test_error = 1-sum(diag(table(sign(Yfit_test),sign(Ytest))))/length(Ytest)
      }
    }
    gl_results$active_nodes = active_nodes_union(gl_results$beta, NODES = NODES)
    gl_results$subgraph_active_edges_rate = NULL
    gl_results$nonzeros_percentage = sum(gl_results$beta!=0)/length(gl_results$beta)
    gl_results$type = "union"
    class(gl_results) <- "graphclass"
    return(gl_results)
  }else{if(type=="fusion"){
    # Fusion penalty -----------------------------------------------------------
    # Check params
    if(is.null(params)){
      beta_start <- rep(0,NODES*(NODES-1)/2)
      b_start <- 0
      MAX_ITER <- 200;    CONV_CRIT <- 1e-04;   MAX_TIME = Inf
    }else{if(is.null(params$beta_start)) {
      beta_start <- rep(0,NODES*(NODES-1)/2)
      b_start <- 0
      MAX_ITER <- params$MAX_ITER;    CONV_CRIT <- params$CONV_CRIT;   MAX_TIME = params$MAX_TIME
    }else{
      beta_start <- params$beta_start*alpha_normalization
      b_start <- params$b_start
      MAX_ITER <- params$MAX_ITER;    CONV_CRIT <- params$CONV_CRIT;   MAX_TIME = params$MAX_TIME
    }}
    print(lambda1)
    print(lambda2)
    gl = logistic_fused_lasso(X, Y, D,
                              lambda1 = lambda1, lambda2 = lambda2,
                              id, verbose, beta_start = beta_start, b_start = b_start,
                              NODES = NODES, MAX_ITER = MAX_ITER, 
                              CONV_CRIT = CONV_CRIT, 
                              MAX_TIME = MAX_TIME)
    gl_results$beta = gl$best_beta/alpha_normalization
    gl_results$b = gl$best_b
    
    # Train fitting
    Yfit <- alpha_normalization*(X%*%gl$best_beta) + gl$b
    gl_results$Yfit <- exp(Yfit)/(1+exp(Yfit))
    gl_results$train_error <- 1- sum(diag(table(sign(Yfit),sign(Y))))/length(Y)
    # Test fitting
    if(!is.null(Xtest)) {
      levels(Ytest) <- c(-1, 1)
      Yfit_test <- Xtest%*%gl_results$beta + gl_results$b
      gl_results$Yfit_test <- exp(Yfit_test)/(1+exp(Yfit_test))
      gl_results$test_error = 1-sum(diag(table(sign(Yfit_test),sign(Ytest))))/length(Ytest)
    }
    gl_results$type = "intersection"
    class(gl_results) <- "graphclass"
    return(gl_results)
  }else{
    stop("The value of type should be one between \"intersection\" and \"union\"")
  }
  }
  }
  }
}




.onLoad <- function(lib, pkg){
  Rdpack::Rdpack_bibstyles(package = pkg, authors = "LongNames")
  invisible(NULL)
}